# Task

In [7]:
# import libraries
import pandas as pd

### EDA (Exploratory Data Analysis)

In [8]:
# load data into pandas DataFrame
df_initial = pd.read_csv('./data/amplitude_test.csv', index_col=0)
df_initial['event_id'] = df_initial.index

df_initial.head(1)

,client_event_time,device_id,session_id,language,country,region,platform,os_name,device_type,device_family,event_type,event_properties,user_properties,event_id
0,2023-10-24 15:42:24.506,84bfeea2-f6ec-47c1-b9d0-791d1ac5c616,1698162134395,English,United States,North Carolina,Web,Chrome,NaN,K,session_end,{},{'fbclid': 'IwAR295tUiGCljtUzVYpJ6BMKwmbhOsfWO...,0


### Построение Воронки (Funnel)  
Для того, чтобы построить воронку конверсии на разных этапах оформления заказа мы выберем для каждой сессии те этапы, на которые вышел пользователь.  
Для этого мы выберем только колонки *session_id* и *event_type* и затем уберем дубликаты. Таким образом мы получим все Этапы для каждой сессии.  
В воронке мы будем считать количество сессий для каждого этапа воронки.

In [9]:
# take columns: session_id, event_type:  
df_funnel = df_initial.copy()
df_funnel = df_funnel[['session_id', 'event_type']]
# drop duplicates:
df_funnel = df_funnel.drop_duplicates()
# save to csv file:
df_funnel.to_csv('./data/df_funnel_session_event.csv', sep='|', index=False)
df_funnel.head(2)

,session_id,event_type
0,1698162134395,session_end
1,1698250740429,session_start


![](./pic/1.png)

В Воронке мы можем отметить следующее:
- В продукте достаточно высокая конверсия в начало использования "Конфигуратора" (более половины всех пользователей, которым показали страницу, запустили конфигуратор). Изходя из этого делаем вывод о том, что сам конфигуратов представляется интересным пользователям
- Конферсия в работу с конфигуратором также достаточно высокая
- Далее мы видим существенное снижение конверсии (только 9,3% пользователей долшщи до Home page магазина)

Продолжим рассмотрение воронки далее:
- В продукте хорошая конверсия из *Home page* в *basket config*
- Также далее коверсия тоже очень хорошая вплоть до *Random button*. Возникает вопрос к функционалу такой кнопки. Возможно она сбивает пользователей. Возможно она стоит слишком поздно в Client Journey или с ней возникают другие проблемы у пользователей. Возможно стоит провести тест, чтобы ее убрать.
- Далее по воронке существенное снижение возникает на этапе перехода в корзину (в целом это логично, но с этим скорее всего есть возможность поработать - сделать напоминание о корзине, подсветить ее или использовать другие методы)  
Таким образом, только 2,8% пользователей доходит до корзины.

![](./pic/4.png)

Сгруппируем некоторый Events и уберем events, которые не относятся к воронке Target action (payment)

![](./pic/5.png)

Мы видим, что Клиентский путь остается достаточно сложным и оставляет много вопросов к необходимости тех или иных этапов в воронке: Random button, product card submitted etc.

### Funnel в различных разрезах

Посмотрим на воронку продукта в различных разрезах:
- Язык (language)
- Страны (country)
- Регион (region)
- OS
- Утройства (device_type)
- device_family


In [10]:
# device type:
df_funnel_device = df_initial.copy()
df_funnel_device = df_funnel_device[['session_id', 'event_type', 'device_type']]
df_funnel_device = df_funnel_device.drop_duplicates()
df_funnel_device.to_csv('./data/df_funnel_device.csv', sep='|', index=False)
df_funnel_device.head(2)

,session_id,event_type,device_type
0,1698162134395,session_end,NaN
1,1698250740429,session_start,NaN


Рассмотрим следующие группы Устройств:  
![](./pic/6.png)

![](./pic/7.png)

Выше мы отметили, что самое существенное падение конверсии происходит на переходе в *Home page viewed*  
Сравним этот этап для различных типов устройств:
- Base Funnel - все пользователи
- Null - устройство неопределено
- Iphone
- Samsung
- Other  

    - Мы можем видеть что самая низкая конверсия на рассматриваемом этапе наблюдается для Iphone и Samsung (остается 6.12% и 5% изначального трафика соответственно, в то время как глобальная цифра 9.26%, то есть существенно выше.)  
    - В категории *NULL*, то есть пользователи, чьи устройства не были идентифицированны, конверсия на этом этапе сопоставима с глобальным средним.
    - Также видно, что наилучшая конверсия в этот этап происходит в категории Other. Куда входят не только трафик мобильных устройств, но и другие. Посмотрим на Other более подробно, чтобы выявить лидеров по конверсии...


![](./pic/8.png)

- Видим, что конверсия среди пользователей других мобильных устройств сопоставима с конверсией пользователей Iphone и Samsung (то есть также низкая)
- В тоже время конверсия среди пользователей Windows, Mac и Linux (то есть Desktop платформ) существенно выше (38.45%).  
  

Изходя из этого можно сделать предположение о том, что для пользователей мобильных устройств на этом этапе возникает какой-то барьер (неверно отображаются элементы и т.п.)  

# Выводы  
Исходя из аналитики воронки продукта можно сделать следующие выводы:
- Наиболее острая "боль" продукта - низкая конверсия в Home Page среди пользователей **мобильных устройств**. Необходимо провести детальный инвестигейт клиентского пути на этом этапе.
- Также возниют вопросы о необходимости всех этапов клиентского пути (возможно стоит провести тесты для проверки гипотез о том, что те или иные элементы воронки являются лишними или неэффективными)

# Доработка задания

- Home page view не является частью воронки клиентского пути, это можно увидеть из схемы ниже:  
![](./pic/9.png)

Изсходя из этой схемы выделим только обязательные этапы воронки:  
1. session_start
2. Configurator start
3. basket config pressed
4. order view
5. payment flow (personal info etc..)

Рассмотрим воронку основных этапов заказа:  
![](./pic/10.png)

Из пользователей, которые попали на сайт, почти 42% пользователей теряется на первом шаге (начало работы с конфигуратором)  
Посмотрим именно на этот этап воронки в разрезе устройств пользователей

![](./pic/11.png)

В целом видим, что на этом этапе срезается более 40% пользователей вне зависимости от типа устройства

Также, внутри Other (куда входят Desktop платформы и другие мобильные устройства) не наблюдается различий на этом этапе воронки  
![](./pic/12.png)

### Languages

![](./pic/13.png)

В разрезе языков пользователей рассмотрим 3 самых растространенных: Анлийский, Немецкий, Русский и остальные объединим как Other   
Можно увидеть, что худшая конверсия в запуск конфигуратора происходит среди пользователей, который используют Русский язык.  
Исходя из этого можно было бы сделать предположение о том, что интерфейс сайта на Русском менее удобен для пользователей.  
Также стоит отметить, что большая часть трафика - это пользователи, использующие Английский язык, однако конверсия среди таких пользователей хуже чем конверсия среди пользователей, исп. Немецкий язык - на это стоит обратить внимание.

## OS groups  
![](./pic/14.png)

- Instagram является лидером по количество пользователей на анализируемом сайте. Положительным является то, что конверсия в Корзину для этой платформы является самым высоким, однако до оплаты тем не менее доходит почти самая низкая доля пользователей относительно других платформ

# Гипотезы

- Работа с конфигуратором ослажнена для пользователей других языков (кроме Английского и Немецкого)
    - Как проверить: взять для эксперимента один пример языка (Русский например), предложить возможные улучшения и затем запустить A/B тест для пользователей, использующих этот язык. Дизайн и интерпретация результатов A/B теста является важным вопросом, здесь отметить только основные моменты - необходимо верно выбрать метрику (наиболее близкую к изменению). Если мы проверяем изменения в конфигураторе, то берем конверсию связанную с ним (из начального экрана в работу с конфигуратором к примеру)
    - В случае если изменения дадут статистически важную разницу для вариаций эксперимента (в лучшую сторону для новой версии конфигуратора на Русском) такое изменение можно перенести для всех пользователей, использующих Русский

- Пользователи из Facebook и Instagram теряются сильнее других на этапе передачи персональных данных
    - Проверка Гипотезы аналогичным образом